In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.cluster import DBSCAN
import ta

In [6]:
df=pd.read_csv("data/btc_futures_data.csv")
df['time'] = pd.to_datetime(df['Open Time']) # 꺼낸 데이터 프레임의 날짜 변수 형변환을 진행합니다.
df.set_index(['time'], inplace = True)
df.drop(['Open Time', 'Close Time'], axis=1, inplace=True)
df.fillna(0, inplace=True)
df

,Open,High,Low,Close,Volume,EMA9,EMA60,EMA200,RSI,RSI_SMA,StochRSI_%K,StochRSI_%D,MACD,MACD_Signal,MACD_Hist
time,,,,,,,,,,,,,,,
2019-09-09 01:00:00,10000.00,10357.53,10000.00,10340.12,471.661,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2019-09-09 05:00:00,10340.12,10412.65,10324.77,10391.63,2624.630,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2019-09-09 09:00:00,10316.62,10316.68,10267.37,10297.89,779.449,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2019-09-09 13:00:00,10297.89,10316.34,10092.71,10149.47,2664.789,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2019-09-09 17:00:00,10149.47,10475.54,10077.22,10414.60,2466.485,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13 01:00:00,86660.00,89830.00,85792.50,88762.80,98262.570,86444.406372,77708.832551,71178.107989,75.643715,81.777815,0.076429,0.304172,3352.754050,2936.653641,416.100409
2024-11-13 05:00:00,88762.90,89920.00,87384.60,87972.00,62526.904,86749.925098,78045.329845,71345.211392,71.367875,80.776868,0.076429,0.144868,3298.688012,3009.060516,289.627497
2024-11-13 09:00:00,87972.00,88619.00,87061.60,87198.20,50628.657,86839.580078,78345.423948,71502.952572,67.355790,78.886755,0.076429,0.076429,3157.009060,3038.650224,118.358835
